In [1]:
import altair as alt
import pandas as pd
import streamlit as st
from pathlib import Path

## Step 1: Import raw data and select regional and global data from 2017 to 2023

In [11]:
# Import raw data
df_raw = pd.read_csv("TB_burden_countries_2025-12-05.csv")
# Step 1a: map g_whoregion 3-letter codes -> readable names
region_map = {
    "AFR": "Africa",
    "AMR": "Americas",
    "EMR": "Eastern Mediterranean",
    "EUR": "Europe",
    "SEAR": "South-East Asia",
    "WPR": "Western Pacific",
}

df = df_raw.copy()

# make sure there is no trailing space
df["g_whoregion"] = df["g_whoregion"].astype(str).str.strip()
df["region"] = df["g_whoregion"].map(region_map)

# keep only rows with a mapped region
df = df[~df["region"].isna()].copy()

# Step 1b: filter to 2017–2023
df = df[(df["year"] >= 2017) & (df["year"] <= 2023)].copy()

## Step 2: Get regional & global incidence rate

In [12]:
# 2a. Regional incidence: average across countries within each region-year
reg_inc = df.groupby(["region", "year"], as_index=False).agg(
    rate=("e_inc_100k", "mean"),
    ci_low=("e_inc_100k_lo", "mean"),
    ci_high=("e_inc_100k_hi", "mean"),
)
reg_inc["level"] = "Regional"
reg_inc["measure"] = "Incidence"

# 2b. Global incidence: average across ALL regions for each year
glob_inc = df.groupby("year", as_index=False).agg(
    rate=("e_inc_100k", "mean"),
    ci_low=("e_inc_100k_lo", "mean"),
    ci_high=("e_inc_100k_hi", "mean"),
)
glob_inc["region"] = "Global"
glob_inc["level"] = "Global"
glob_inc["measure"] = "Incidence"


## Step 3: Get regional & global mortality rate

In [13]:
# 3a. Regional mortality: average across countries within each region-year
reg_mort = df.groupby(["region", "year"], as_index=False).agg(
    rate=("e_mort_100k", "mean"),
    ci_low=("e_mort_100k_lo", "mean"),
    ci_high=("e_mort_100k_hi", "mean"),
)
reg_mort["level"] = "Regional"
reg_mort["measure"] = "Mortality"

# 3b. Global mortality: average across ALL regions (all rows) for each year
glob_mort = df.groupby("year", as_index=False).agg(
    rate=("e_mort_100k", "mean"),
    ci_low=("e_mort_100k_lo", "mean"),
    ci_high=("e_mort_100k_hi", "mean"),
)
glob_mort["region"] = "Global"
glob_mort["level"] = "Global"
glob_mort["measure"] = "Mortality"


## Step 4: Combine and save file 

In [14]:
# 4a. Stack incidence parts together
incidence = pd.concat([reg_inc, glob_inc], ignore_index=True)

# 4b. Stack mortality parts together
mortality = pd.concat([reg_mort, glob_mort], ignore_index=True)

# 4c. Combine incidence + mortality into one tidy table
tb_trends = pd.concat([incidence, mortality], ignore_index=True)

# Keep only needed columns, sort them
tb_trends = tb_trends[["year", "region", "level", "measure", "rate", "ci_low", "ci_high"]]
tb_trends = tb_trends.sort_values(["region", "measure", "year"])

# 4d. Save to CSV for the Streamlit app
tb_trends.to_csv("tb_trends_clean.csv", index=False)

print("Saved cleaned data to tb_trends_clean.csv")
print(tb_trends.head(12))

Saved cleaned data to tb_trends_clean.csv
    year  region     level    measure        rate      ci_low     ci_high
0   2017  Africa  Regional  Incidence  237.744681  145.148936  408.510638
1   2018  Africa  Regional  Incidence  227.234043  141.191489  386.234043
2   2019  Africa  Regional  Incidence  217.361702  136.695745  366.063830
3   2020  Africa  Regional  Incidence  206.968085  129.829787  346.617021
4   2021  Africa  Regional  Incidence  201.891489  125.451064  338.212766
5   2022  Africa  Regional  Incidence  198.723404  122.468085  333.446809
6   2023  Africa  Regional  Incidence  193.553191  117.936170  326.617021
42  2017  Africa  Regional  Mortality   63.042553   40.797872   90.714894
43  2018  Africa  Regional  Mortality   57.327660   37.419149   81.865957
44  2019  Africa  Regional  Mortality   52.061702   34.369787   73.868085
45  2020  Africa  Regional  Mortality   51.101489   33.172553   73.239574
46  2021  Africa  Regional  Mortality   47.250426   30.302340   67.933